In [17]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [18]:
#pip install implicit

In [19]:
website_url = '/Users/rezigibtissam/Downloads/KaDo.csv'
retail_data = pd.read_csv(website_url) # This may take a couple minutes

In [20]:
retail_data['LIBELLE'].nunique()

1484

In [21]:
# Check to see if there are any missing values in the data.
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7245335 entries, 0 to 7245334
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   TICKET_ID   int64  
 1   MOIS_VENTE  int64  
 2   PRIX_NET    float64
 3   FAMILLE     object 
 4   UNIVERS     object 
 5   MAILLE      object 
 6   LIBELLE     object 
 7   CLI_ID      int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 442.2+ MB


In [59]:
item_lookup['LIBELLE'] = retail_data['LIBELLE'].drop_duplicates() # Only get unique item/description pairs
item_lookup = retail_data.drop(columns=['CLI_ID', 'UNIVERS', 'MAILLE', 'MOIS_VENTE', 'TICKET_ID', 'PRIX_NET', 'FAMILLE'])
item_lookup['LibelleCode'] = item_lookup.LIBELLE.map(hash)
#item_lookup.loc[item_lookup['Stock'] < 0, 'LibelleCode'] = item_lookup['Stock'] * -1 
#item_lookup.loc[item_lookup['Stock'] > 0, 'LibelleCode'] = item_lookup['Stock'] * 1
#item_lookup.drop(columns=['Stock'])
item_lookup

,LIBELLE,Stock,LibelleCode
0,GD JDM4 PAMPLEMOUSSE FL 200ML,-7133762948432355791,-7133762948432355791
1,GD JDM4 PAMPLEMOUSSE FL 200ML,-7133762948432355791,-7133762948432355791
2,CR JR PARF BIO.SPE AC.SENT.50ML,-9112326361858664797,-9112326361858664797
3,EAU MICELLAIRE 3 THES FL200ML,3055909093077009594,3055909093077009594
4,GD JDM4 TIARE FL 200ML,-4750610258852243962,-4750610258852243962
...,...,...,...
7245330,SVC CR MAINS ET ONGLES T75ml,1712006776951436622,1712006776951436622
7245331,GD JDM4 LAVANDIN DE PROVENCE 200ML,-1004834416420630274,-1004834416420630274
7245332,CRAYON REGARD PRUNE CN3 1.3G,-5242631876571511823,-5242631876571511823
7245333,MASCARA SOIN NOIR01 2014 CN3 9ML,-3100191199198090619,-3100191199198090619


In [54]:
 item_lookup = retail_data.drop(columns=['CLI_ID', 'UNIVERS'])
 item_lookup = item_lookup['LIBELLE'].drop_duplicates() # Only get unique item/description pairs
 item_lookup = item_lookup.reset_index()
 item_lookup.insert(0, 'StockCode', range(1, 1 + len(item_lookup)))
 del item_lookup['index']
 item_lookup

,StockCode,LIBELLE
0,1,GD JDM4 PAMPLEMOUSSE FL 200ML
1,2,CR JR PARF BIO.SPE AC.SENT.50ML
2,3,EAU MICELLAIRE 3 THES FL200ML
3,4,GD JDM4 TIARE FL 200ML
4,5,EDT UN MATIN AU JARDIN 100ML MUGUET
...,...,...
1479,1480,VOILE TT LUM3 BEIGE SOLEIL T30ML
1480,1481,BIJOU PEAU FLEURS DOREES A/T MAI11 LU3
1481,1482,VAO ROSE A L'EAU A/T MAI10 LUM3 3ML
1482,1483,PAL YX AMB/CANN/CHAT NOEL11 A/T LU3 3X1G


In [24]:
retail_data['Stock'] = retail_data.LIBELLE.map(hash)
grouped_purchased = retail_data.groupby(['CLI_ID', 'LIBELLE']).agg({'LIBELLE':list, 'CLI_ID': max, 'Stock' : max})
grouped_purchased['Quantity'] = grouped_purchased['LIBELLE'].str.len()
grouped_purchased['LibelleCode'] = grouped_purchased['Stock']
del grouped_purchased['LIBELLE']
del grouped_purchased['Stock']
grouped_purchased

CLI_ID  Quantity  \
CLI_ID    LIBELLE                                                       
1490281   CR JR PARF BIO.SPE AC.SENT.50ML           1490281         1   
          EAU MICELLAIRE 3 THES FL200ML             1490281         1   
          GD JDM4 PAMPLEMOUSSE FL 200ML             1490281         2   
          GD JDM4 TIARE FL 200ML                    1490281         1   
13290776  EDT  UN MATIN AU JARDIN  100ML  MUGUET   13290776         1   
...                                                     ...       ...   
997385337 VAO BRIL ROSE SOMPTUEUX 14 CN3 5.5ML    997385337         1   
          VAO BRIL ROUG/SIENN 33 AX/SO CN3 5,5ML  997385337         1   
          VAO HIBISCUS ROUGE ETE13 ANI LU4 3ML    997385337         1   
          VAO PASTEL PARME 03 MANUC CN3 5.5ML     997385337         1   
          ZERO TRACE SPRAY SPF 15 FL150           997385337         1   

                                                          LibelleCode  
CLI_ID    LIBELLE                                                      
1490281   CR JR PARF BIO.SPE AC.SENT.50ML        -9112326361858664797  
          EAU MICELLAIRE 3 THES FL200ML           3055909093077009594  
          GD JDM4 PAMPLEMOUSSE FL 200ML          -7133762948432355791  
          GD JDM4 TIARE FL 200ML                 -4750610258852243962  
13290776  EDT  UN MATIN AU JARDIN  100ML  MUGUET -5518498499991517237  
...                                                               ...  
997385337 VAO BRIL ROSE SOMPTUEUX 14 CN3 5.5ML    -297864513930136310  
          VAO BRIL ROUG/SIENN 33 AX/SO CN3 5,5ML  9053088783660529606  
          VAO HIBISCUS ROUGE ETE13 ANI LU4 3ML    2551726697489218208  
          VAO PASTEL PARME 03 MANUC CN3 5.5ML     5138144477786372220  
          ZERO TRACE SPRAY SPF 15 FL150           -563748396958086268  

[5910987 rows x 3 columns]

In [25]:
customers = list(np.sort(grouped_purchased.CLI_ID.unique())) # Get our unique customers
products = list(grouped_purchased.LibelleCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CLI_ID.astype('category', customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.LibelleCode.astype('category', products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
purchases_sparse

<853484x1484 sparse matrix of type '<class 'numpy.int64'>'
	with 5910987 stored elements in Compressed Sparse Row format>

In [26]:
#We have 7 customers with 24 items. For these user/item interactions, 24 of these items had a purchase. In terms of sparsity of the matrix, that makes:

In [27]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.53330760990174

In [28]:
#Creating a Training and Validation Set

In [29]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [35]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [36]:
#Implementing ALS for Implicit Feedback

In [37]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [39]:
#user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1, rank_size = 20)

In [42]:
# user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

In [41]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


In [43]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [44]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [ ]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

In [45]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [46]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    # return item_lookup.loc[item_lookup.Stock.isin(prod_codes)]
    return prod_codes

In [47]:
customers_arr[:5]

array([ 1490281, 13290776, 20163348, 20200041, 20561854])

In [55]:
get_items_purchased(1490281, product_train, customers_arr, products_arr, item_lookup)

array([-4822006058930556278, -8393204533579971964,  1554969629711514997])

In [57]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.LIBELLE.loc[item_lookup.LibelleCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    LIBELLE = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'Stock': codes, 'LIBELLE': LIBELLE}) # Create a dataframe 
    return final_frame[['Stock', 'LIBELLE']] # Switch order of columns around

In [60]:
rec_items(1490281, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,Stock,LIBELLE
0,-576714536581527494,BAUME POINTES SECHES 50ML
1,3385727460688505702,DEO JDM4 COTON FL50ML
2,-2302481396267097759,SVC PIEDS PAST EFF RELAX
3,7839377566862806438,"RAL PECHE 21 LUMINELLE4 3,5G"
4,-4221301824572807022,PERLES SOLEIL ETE13 ANI LU4 28G
5,-4127421564642774551,"FAP MONO 2013 CN3 2,5G BLEU AZUR"
6,-5390563201626001888,FDT RAYON JEUNESSE BEIGE300 CN3 30ML
7,7647713933822037198,GEL DOUCEUR MAINS HAMAM FL300ML
8,3631496608962659411,DEO JDM4 AMANDE FL 50 ML
9,-6344200568912636768,"RAL POMME D'AMOUR 31 LUM4 3,5G"
